In [ ]:
#@title

#@markdown <center><h3>Jellyfin<br />The Free Software Media System.</h3></center><br>
import os, zipfile, tarfile, requests, re
import urllib.request
from IPython.display import clear_output, display
from ipywidgets import Dropdown, Select
from subprocess import Popen
import xml.etree.ElementTree as ET


#####################################
USE_FREE_TOKEN = True  # @param {type:"boolean"}
TOKEN = ""  # @param {type:"string"}
LATEST_VERSION = False  # @param {type:"boolean"}
CUSTOM_VERSION = "10.8.0"  # @param {type:"string"}
#@markdown <small>&emsp;Version are available on https://repo.jellyfin.org/releases/server/linux/versions/stable/combined/</small>
REGION = "SA" #@param ["US", "EU", "AP", "AU", "SA", "JP", "IN"]
PORT_FORWARD = "argotunnel" #@param ["ngrok", "localhost", "argotunnel"]
#@markdown Default <br>&emsp;username : admin

HOME = os.path.expanduser("~")

if not os.path.exists(f"{HOME}/.ipython/ocr.py"):
    hCode = "https://raw.githubusercontent.com/biplobsd/" \
                "OneClickRun/master/res/ocr.py"
    urllib.request.urlretrieve(hCode, f"{HOME}/.ipython/ocr.py")


from ocr import (
    runSh,
    loadingAn,
    PortForward_wrapper,
    findProcess,
    CWD,
    textAn
)

loadingAn()

os.makedirs('tools/jellyfin/temp', exist_ok=True)
os.makedirs('tools/jellyfin/transcodes', exist_ok=True)
os.makedirs('downloads', exist_ok=True)

def ff_loader(name):
  import requests

  url = requests.get(
    "https://ffbinaries.com/api/v1/version/latest"
    ).json()['bin']['linux-64'][name]
  urllib.request.urlretrieve(url, f'tools/jellyfin/L{name}.zip')

  with zipfile.ZipFile(f'tools/jellyfin/L{name}.zip', 'r') as zip_ref:
      zip_ref.extractall('tools/jellyfin/')
  os.remove(f"tools/jellyfin/L{name}.zip")
  runSh("rm -rf tools/jellyfin/__MACOSX")
  os.replace(f"tools/jellyfin/{name}", f"/usr/bin/{name}")
  os.chmod(f"/usr/bin/{name}", 0o755)

def customVersionSelect():
  allVersionPage = requests.get("https://repo.jellyfin.org/releases/server/linux/versions/stable/combined/").text
  allVersionTags = re.findall(r'">(\d+\.\d+\.\d+)', allVersionPage, re.M)
  option_list = sorted(allVersionTags, reverse=True)
  dropdown = Select(description="Choose one:", options=option_list)
  dropdown.observe(main, names='value')
  display(dropdown)

def startService():
  # Starting jellyfin
  if not findProcess("jellyfin", "--ffmpeg"):
    CM = r"tools/jellyfin/jellyfin/jellyfin --service " \
    f"-C tools/jellyfin -d {CWD}/tools/jellyfin"
    Popen(CM.split(), shell=True)


  # START_SERVER
  # Ngrok region 'us','eu','ap','au','sa','jp','in'
  Server = PortForward_wrapper(
      PORT_FORWARD, TOKEN, USE_FREE_TOKEN, [['jellyfin', 8096, 'http']],
      REGION.lower(), [f"{HOME}/.ngrok2/jellyfin.yml", 7097]
  ).start('jellyfin', displayB=True)

def main(version=None):

  # JellyfinMain
  if LATEST_VERSION:
    serverUrl = "https://repo.jellyfin.org/releases/server/linux/stable/combined/"
  elif CUSTOM_VERSION:
    serverUrl = f"https://repo.jellyfin.org/releases/server/linux/versions/stable/combined/{CUSTOM_VERSION}/"
  else:
    serverUrl = f"https://repo.jellyfin.org/releases/server/linux/versions/stable/combined/{version.new}/"

  server = requests.get(serverUrl).text
  server_file = re.search(r'f=\"(.+)\">jellyfin_(.+)_([^-]+)\.tar\.gz<',
                              server, re.MULTILINE)
  server_file_url = server_file.group(1)
  versionTag = server_file.group(2)

  clear_output(True)
  loadingAn()
  textAn(f"Installing v{versionTag} ...", ty='twg')

  outputFile = 'tools/jellyfin/temp/jellyfin.tar.gz'
  urllib.request.urlretrieve(serverUrl+server_file_url, outputFile)
  with tarfile.open(outputFile, 'r:gz') as tar_ref:
      tar_ref.extractall('tools/jellyfin')
  os.renames(f'tools/jellyfin/jellyfin_{versionTag}',
            'tools/jellyfin/jellyfin')
  os.remove(outputFile)

  # Default config data
  defaultConfig = "https://raw.githubusercontent.com/biplobsd" \
  "/OneClickRun/master/res/appdata/jellyfin/data.tar.gz"
  outputFile = 'tools/jellyfin/temp/data.tar.gz'
  urllib.request.urlretrieve(defaultConfig, outputFile)
  with tarfile.open(outputFile, 'r:gz') as tar_ref:
      tar_ref.extractall('tools/jellyfin')
  os.remove(outputFile)

  # https://github.com/rokibulislaam/colab-ffmpeg-cuda
  ## with GPU
  if os.system("nvidia-smi --query-gpu=name --format=csv,noheader")==0:
    encoConfigP = 'tools/jellyfin/config/encoding.xml'
    mytree = ET.parse(encoConfigP)
    myroot = mytree.getroot()

    def addnew(name, value):
      ET.SubElement(myroot, name).text = value

    addnew('TranscodingTempPath', '/content/tools/jellyfin/transcodes')
    addnew('HardwareAccelerationType', 'nvenc')
    addnew('DeinterlaceMethod', 'yadif')
    addnew('EnableDecodingColorDepth10Hevc', 'true')
    addnew('EnableDecodingColorDepth10Vp9', 'true')
    addnew('EnableDecodingColorDepth10Vp9', 'true')
    addnew('EncoderPreset', 'slower')
    addnew('EncodingThreadCount', '0')

    for tagname in myroot.iter('HardwareDecodingCodecs'):
      ET.SubElement(tagname, 'string').text = 'hevc'
      ET.SubElement(tagname, 'string').text = 'mpeg2video'
      ET.SubElement(tagname, 'string').text = 'mpeg4'
      ET.SubElement(tagname, 'string').text = 'vp8'
      ET.SubElement(tagname, 'string').text = 'vp9'

    mytree.write(encoConfigP)

    runSh(
        "git clone --depth 1 --branch main https://github.com/rokibulislaam/colab-ffmpeg-cuda.git tools/colab-ffmpeg-cuda")
    runSh("cp -r tools/colab-ffmpeg-cuda/bin/. /usr/bin/")
    ffmpegPath = '/usr/bin/ffmpeg'
  else:
    ## without GPU
    # ffmpeg
    ff_loader('ffmpeg')

    # ffprobe
    ff_loader('ffprobe')

    ffmpegPath = '/usr/bin/ffmpeg'

  startService()

if not os.path.exists("tools/jellyfin/jellyfin/jellyfin"):
  if LATEST_VERSION or CUSTOM_VERSION:
    main()
  else:
    customVersionSelect()
else:
  startService()


In [ ]:
import time

def keep_colab_running():
    while True:
       # print("Colab is still running")
        time.sleep(60)  # Sleep for 60 seconds

keep_colab_running()
